In [1]:
import multiprocessing, torch, re
import os.path as osp
import datasets

from transformers import EncoderDecoderModel, BertTokenizerFast,\
    Seq2SeqTrainer, Seq2SeqTrainingArguments
from tokenization_kobert import KoBertTokenizer
from datasets import load_dataset, Dataset
from evaluate import load as load_metric
from tqdm.auto import tqdm
from time import strftime, time, localtime
from os import listdir
from functools import partial
from typing import List
from zipfile import ZipFile

from utils import rouge, load_dataset

print(torch.__version__)

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


1.12.1+cu113


In [2]:
# model_path = "skt/kogpt2-base-v2"
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path, 
#                                                     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#                                                     pad_token='<pad>', mask_token='<mask>', max_length=512)
# model = GPT2LMHeadModel.from_pretrained(model_path)
# model = BertForMaskedLM.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained('monologg/distilkobert')
# model = BertForMaskedLM.from_pretrained("monologg/kobert-lm")
# tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')
model = EncoderDecoderModel.from_pretrained("output/checkpoint-72000")
tokenizer = BertTokenizerFast.from_pretrained("output/checkpoint-72000")
# model

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [3]:
# mask_idxs = encoded['input_ids'] == tokenizer.mask_token_id
# print(mask_idxs)
# print(out.logits.shape)
# masks = out.logits[mask_idxs].argmax(dim=-1)
# print(masks)
# print(tokenizer.decode(masks))
# predicted_token = out.logits[0,-1].argmax(-1)
# tokenizer.decode(predicted_token)

In [4]:
# dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
# dataset = load_dataset("wikipedia", "20220301.en")
# dataset = dataset.remove_columns([col for col in dataset.column_names if col != 'text'])
# dataset = load_dataset("bookcorpus", split="train", streaming=True).with_format('torch')
# dataset = load_dataset("bookcorpus", split="train")
# dataset

In [5]:
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(dataset['train']), batch_size)):
#         yield dataset['train'][i:i+batch_size]['text']
# if [_dir for _dir in listdir() if "tokenizer" in _dir] != []:
#     latest_tokenizer_path = sorted([_dir for _dir in listdir() if "tokenizer" in _dir])[-1]
#     tokenizer = AutoTokenizer.from_pretrained(latest_tokenizer_path)
# else:
#     # tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer = AutoTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer.save_pretrained(get_time_dir())

# tokenizer.pad_token = '<pad>'
# tokenizer.eos_token = '</s>'
# tokenizer.bos_token = '</s>'
# tokenizer.unk_token = '<unk>'
# tokenizer.mask_token = '<mask>'
# tokenizer.model_max_length = 32

In [6]:
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 512


In [7]:
def get_time_dir(): return f"{strftime('%m-%d-%H-%M', localtime(time()))}"

In [8]:
from utils import load_iter_train_test
# dataset = load_dataset(tokenizer=tokenizer)
trainset, testset = load_iter_train_test(tokenizer=tokenizer)

In [9]:
# one = dataset['train'][0]
# one

In [10]:
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.config.max_new_tokens = 16
model.config.early_stopping = True
# model.config.no_repeat_ngram_size = 1
model.config.length_penalty = 2.
model.config.repetition_penalty = 3.
model.config.num_beams = 10
model.config.vocab_size = model.config.encoder.vocab_size


In [11]:
sent = "안녕ㅎ"
"안녕하세요"

'안녕하세요'

In [12]:
encoded = tokenizer(sent, return_tensors="pt")

In [13]:
encoded

{'input_ids': tensor([[    2, 15655,  8509,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [14]:
out = model.generate(input_ids=encoded['input_ids'], attention_mask=encoded['attention_mask'])[0]
tokenizer.decode(out)

'[CLS] 할 [CLS] [CLS] 잘 [CLS] 다 [CLS] 더 [CLS] 안 [CLS] 좀 [CLS] 거 [CLS] 것 [CLS] 나 [CLS] [UNK] [CLS] 수 [CLS] 제일 [CLS] 한 [CLS] 본 [CLS] 많이 [CLS]'

In [16]:
out

tensor([    2,  7657,     2,     2,  5957,     2,  3110,     2,  3175,     2,
         5508,     2,  6096,     2,  2173,     2,  2190,     2,  2783,     2,
            1,     2,  5099,     2, 14530,     2,  7653,     2,  4489,     2,
        14011,     2])